In [30]:
#!pip install semanticscholar

In [7]:
import os
from dotenv import load_dotenv, get_key
load_dotenv()

True

In [5]:
from langchain.agents import Tool
from langchain_community.tools.semanticscholar.tool import SemanticScholarAPIWrapper

semantic_scholar = SemanticScholarAPIWrapper()

tools = []

tools.append(
   Tool(
       name="Semantic Scholar Search",
       func=semantic_scholar.run,
       description="Useful for retrieving academic references, citations, and publications.",
   )
)

print(tools)


[Tool(name='Semantic Scholar Search', description='Useful for retrieving academic references, citations, and publications.', func=<bound method SemanticScholarAPIWrapper.run of SemanticScholarAPIWrapper(semanticscholar_search=<bound method SemanticScholar.search_paper of <semanticscholar.SemanticScholar.SemanticScholar object at 0x11465d5d0>>, top_k_results=5, S2_MAX_QUERY_LENGTH=300, load_max_docs=100, doc_content_chars_max=4000, returned_fields=['title', 'abstract', 'venue', 'year', 'paperId', 'citationCount', 'openAccessPdf', 'authors', 'externalIds'])>)]


In [36]:
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import StructuredTool

def chunkify(text: str, max_length: int) -> list[str]:
    """Split the text into manageable chunks."""
    sentences = text.split(". ")
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence) + 2
        if current_length + sentence_length > max_length:
            chunks.append(". ".join(current_chunk) + ".")
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += sentence_length

    if current_chunk:
        chunks.append(". ".join(current_chunk) + ".")

    return chunks

def create_reference(publications: str) -> str:
    """Prepare a list of IEEE bibliograpy formatter reference from given list of publications."""

    prompt_extract = (
        "You are an assistant tasked with creating IEEE styled referece for citation from provided publications. "
        "Now, please generate IEEE bibliography styled references for following publications.\n\n"
        "Publications: {publications}"
    )

    prompt = PromptTemplate.from_template(prompt_extract)
    chain = prompt | llm
    processed_chunks = []
    chunks = chunkify(publications, 2000)
    for chunk in chunks:
        result = chain.invoke({"publications": chunk})
        processed_chunks.append(result.content)

    return "\n".join(processed_chunks)

reference_tool = Tool(
    name="Reference Creator",
    description="Create IEEE conventional reference for citation from a list of academic publications.",
    func=create_reference
)

tools.append(reference_tool)

print(tools)

[Tool(name='Semantic Scholar Search', description='Useful for retrieving academic references, citations, and publications.', func=<bound method SemanticScholarAPIWrapper.run of SemanticScholarAPIWrapper(semanticscholar_search=<bound method SemanticScholar.search_paper of <semanticscholar.SemanticScholar.SemanticScholar object at 0x11465d5d0>>, top_k_results=5, S2_MAX_QUERY_LENGTH=300, load_max_docs=100, doc_content_chars_max=4000, returned_fields=['title', 'abstract', 'venue', 'year', 'paperId', 'citationCount', 'openAccessPdf', 'authors', 'externalIds'])>), Tool(name='Reference Creator', description='Create IEEE conventional reference for citation from a list of academic publications.', func=<function create_reference at 0x124326fc0>), Tool(name='Reference Creator', description='Create IEEE conventional reference for citation from a list of academic publications.', func=<function create_reference at 0x12480b100>), Tool(name='Reference Creator', description='Create IEEE conventional re

In [37]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=os.getenv('GEMINI_API_KEY'),
    temperature=0.1
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    max_iterations=3,
    handle_parsing_errors=True,
    verbose=True,
)

In [41]:
chat_history = []
user_input = """
1. Find related publications for on-device LLM Quantization.
2. Create references from the publications found.
"""
response = agent.run(input=user_input, chat_history=chat_history)
chat_history.append(("User: " + user_input, "Agent: " + response))

print("Final Content for Query:\n", response)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Semantic Scholar Search
Action Input: on-device LLM Quantization
Observation: Published year: 2023
Title: AWQ: Activation-aware Weight Quantization for On-Device LLM Compression and Acceleration
Authors: Ji Lin, Jiaming Tang, Haotian Tang, Shang Yang, Xingyu Dang, Song Han
Abstract: Large language models (LLMs) have transformed numerous AI applications. On-device LLM is becoming increasingly important: running LLMs locally on edge devices can reduce the cloud computing cost and protect users' privacy. However, the astronomical model size and the limited hardware resource pose significant deployment challenges. We propose Activation-aware Weight Quantization (AWQ), a hardware-friendly approach for LLM low-bit weight-only quantization. AWQ finds that not all weights in an LLM are equally important. Protecting only 1% salient weights can greatly reduce quantization error. To identify salient weight chann